# Data Wrangling with DataFrames Coding Quiz

Use this Jupyter notebook to find the answers to the quiz in the previous section. There is an answer key in the next part of the lesson.

In [2]:
from pyspark.sql import SparkSession

# TODOS: 
# 1) import any other libraries you might need
# 2) instantiate a Spark session 
# 3) read in the data set located at the path "data/sparkify_log_small.json"
# 4) write code to answer the quiz questions 
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import desc
from pyspark.sql.functions import asc
from pyspark.sql.functions import sum as Fsum

import datetime

import numpy as np
import pandas as pd
spark = SparkSession.builder.appName('Data Wrangling').getOrCreate()

In [4]:
path = "data/sparkify_log_small.json"
user_log = spark.read.json(path)
user_log.printSchema()

root
 |-- artist: string (nullable = true)
 |-- auth: string (nullable = true)
 |-- firstName: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- itemInSession: long (nullable = true)
 |-- lastName: string (nullable = true)
 |-- length: double (nullable = true)
 |-- level: string (nullable = true)
 |-- location: string (nullable = true)
 |-- method: string (nullable = true)
 |-- page: string (nullable = true)
 |-- registration: long (nullable = true)
 |-- sessionId: long (nullable = true)
 |-- song: string (nullable = true)
 |-- status: long (nullable = true)
 |-- ts: long (nullable = true)
 |-- userAgent: string (nullable = true)
 |-- userId: string (nullable = true)



In [5]:
user_log.take(5)

[Row(artist='Showaddywaddy', auth='Logged In', firstName='Kenneth', gender='M', itemInSession=112, lastName='Matthews', length=232.93342, level='paid', location='Charlotte-Concord-Gastonia, NC-SC', method='PUT', page='NextSong', registration=1509380319284, sessionId=5132, song='Christmas Tears Will Fall', status=200, ts=1513720872284, userAgent='"Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/36.0.1985.125 Safari/537.36"', userId='1046'),
 Row(artist='Lily Allen', auth='Logged In', firstName='Elizabeth', gender='F', itemInSession=7, lastName='Chase', length=195.23873, level='free', location='Shreveport-Bossier City, LA', method='PUT', page='NextSong', registration=1512718541284, sessionId=5027, song='Cheryl Tweedy', status=200, ts=1513720878284, userAgent='"Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/36.0.1985.143 Safari/537.36"', userId='1000'),
 Row(artist='Cobra Starship Featuring Leighton Meester', auth='Logged In', firstNa

# Question 1

Which page did user id "" (empty string) NOT visit?

In [6]:
# TODO: write your code to answer question 1
user_log.select("page").where(user_log.userId == "").dropDuplicates().show()

+-----+
| page|
+-----+
| Home|
|About|
|Login|
| Help|
+-----+



# Question 2 - Reflect

What type of user does the empty string user id most likely refer to?


In [7]:
# TODO: use this space to explore the behavior of the user with an empty string
user_log.select(['userId', 'registration']).where(user_log.userId == "").show()

+------+------------+
|userId|registration|
+------+------------+
|      |        null|
|      |        null|
|      |        null|
|      |        null|
|      |        null|
|      |        null|
|      |        null|
|      |        null|
|      |        null|
|      |        null|
|      |        null|
|      |        null|
|      |        null|
|      |        null|
|      |        null|
|      |        null|
|      |        null|
|      |        null|
|      |        null|
|      |        null|
+------+------------+
only showing top 20 rows



# Question 3

How many female users do we have in the data set?

In [8]:
# TODO: write your code to answer question 3
female_user = user_log.select(['userId', 'gender']).dropDuplicates().groupby(user_log.gender == 'F').count().show()

+------------+-----+
|(gender = F)|count|
+------------+-----+
|        null|    1|
|        true|  462|
|       false|  501|
+------------+-----+



# Question 4

How many songs were played from the most played artist?

In [9]:
# TODO: write your code to answer question 4
most_played_artist_songs = user_log.select(['artist','song']).groupby(user_log.artist).count().sort(desc('count')).show()

+--------------------+-----+
|              artist|count|
+--------------------+-----+
|                null| 1653|
|            Coldplay|   83|
|       Kings Of Leon|   69|
|Florence + The Ma...|   52|
|            BjÃÂ¶rk|   46|
|       Dwight Yoakam|   45|
|       Justin Bieber|   43|
|      The Black Keys|   40|
|         OneRepublic|   37|
|                Muse|   36|
|        Jack Johnson|   36|
|           Radiohead|   31|
|        Taylor Swift|   29|
|Barry Tuckwell/Ac...|   28|
|          Lily Allen|   28|
|               Train|   28|
|           Daft Punk|   27|
|           Metallica|   27|
|          Nickelback|   27|
|          Kanye West|   26|
+--------------------+-----+
only showing top 20 rows



# Question 5 (challenge)

How many songs do users listen to on average between visiting our home page? Please round your answer to the closest integer.



In [14]:
# TODO: write your code to answer question 5
home_page_event = udf(lambda x: 1 if x == 'Home' else 0, IntegerType())

In [24]:
user_log = user_log.withColumn('Home_Page', home_page_event('page'))

In [45]:
from pyspark.sql import Window
windowval = Window.partitionBy('userId').orderBy('ts').rangeBetween(Window.unboundedPreceding,0)

In [46]:
user_log = user_log.withColumn('phase', Fsum('Home_Page').over(windowval))

In [60]:
user_log.filter((user_log.page == 'NextSong')).groupBy('userID', 'phase').agg({'phase':'count'}).agg({'count(phase)':'avg'}).show()

+------------------+
| avg(count(phase))|
+------------------+
|6.9558333333333335|
+------------------+

